In [67]:
import csv

In [68]:
file = "data/skos/cv_burial.csv"

In [69]:
class Csv2SkosReader(object):
    """
    extract SKOS-like objects from special structured CSV sheets
    """
            
    def __init__(self, csv_file):
        self.csv_file = csv_file
        try:
            with open(self.csv_file, encoding='utf-8') as csvfile:
                self.data = [x for x in csv.reader(csvfile, delimiter=',')]
        except:
            print('could not open csv file')
        self.headers = self.data[0]
        try:
            self.alt_lang = (self.headers[1])[(self.headers[1]).index('@')+1:]
        except:
            self.alt_lang = None
        self.schemes = set([x[0] for x in self.data[1:]])
        self.number_of_schemes = len(self.schemes)
        
    def get_concepts(self):
        concepts = [] 
        for x in self.data[1:]:
            first_order = x[1].split('|')
            if x[2] != '':
                second_order = x[2].split('|')
                concept = {
                    'scheme': x[0],
                    'concept': {
                        'pref_label': first_order[0],
                        'pref_label_lang': 'eng',
                        'alt_label': self.alt_lang,
                        'alt_label_lang': self.alt_lang,
                        'narrower':{
                            'scheme': x[0],
                            'concept': {
                                'pref_label': second_order[0],
                                'pref_label_lang': 'eng',
                                'alt_label': second_order[1],
                                'alt_label_lang': self.alt_lang,
                            }
                        }
                    }
                }
            else:
                concept = {
                    'scheme': x[0],
                    'concept': {
                        'pref_label': first_order[0],
                        'pref_label_lang': 'eng',
                        'alt_label': first_order[1],
                        'alt_label_lang': self.alt_lang,
                    }
                }
            concepts.append(concept)
            
        return concepts         

In [101]:
class Csv2SkosImporter(Csv2SkosReader):
    """Takes a special formatted csv file, parses it and imports the derived data into vocabs"""
    
    def update_schemes(self):
        """import/updates all conceptSchemes found in csv"""
        report = {}
        report['before'] = len(SkosConceptScheme.objects.all())
        failed = []
        success = []
        for x in self.schemes:
            try:
                clean = x.split('|')[0].strip()
            except:
                clean = x.strip()
            try:
                temp_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title=clean)
                temp_scheme.save()
                success.append(x)
            except:
                failed.append(x)
        report['failed'] = failed
        report['success'] = success
        report['after'] = len(SkosConceptScheme.objects.all())
        return report
    
    def update_concepts(self):
        """import/updates all SkosConcepts found in csv"""
        report = {}
        report['before'] = len(SkosConcept.objects.all())
        failed = []
        success = []
        for x in self.get_concepts():
            #print(x['concept'])
            pass
        report['after'] = len(SkosConcept.objects.all())
        return report

In [102]:
hansi = Csv2SkosImporter('data/skos/cv_burial.csv')

In [103]:
hansi.update_concepts()

{'after': 0, 'before': 0}

In [66]:
for x in SkosConceptScheme.objects.all():
    x.delete()

In [12]:
readCSV

NameError: name 'readCSV' is not defined

In [20]:
readCSV

In [63]:
string = 'hansi@ger'

In [64]:
string.index('@')

5